# Tensorflow 2 Logistic Regression

Partly derived from tutorials by Aymeric Damien (https://github.com/aymericdamien/TensorFlow-Examples/)

In [ ]:
# import tensorflow module. Check API version.
import tensorflow as tf

print (tf.__version__)

# required for TF to run within docker using GPU (ignore otherwise)
gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

## Load the data

### MNIST Dataset - Handwritten Digits (0-9)

![MNIST Sample Data](images/MnistExamples.png)

In [ ]:
# download the MNIST data
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
# MNIST dataset parameters
num_classes = 10 # 0 to 9 digits
num_features = 784 # 28*28

In [ ]:
import numpy as np

# convert to float32
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

# flatten images to 1-D vector of 784 features (28*28)
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])

# normalize images value from [0, 255] to [0.0, 1.0]
x_train, x_test = x_train / 255., x_test / 255.

## Define Model Equations

In [ ]:
# logistic regression model equation y = sigmoid(WX + b)
def logistic_regression(X, W, b):
    # apply softmax to normalize the logits to a probability distribution.
    return tf.nn.softmax(tf.add(tf.matmul(X, W), b))

# cross-entropy loss function
def cross_entropy(y_pred, y_true):
    # encode label to a one hot vector
    y_true = tf.one_hot(y_true, depth=num_classes)
    # clip prediction values to avoid log(0) error
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    # compute cross-entropy
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred), 1))

# accuracy metric
def accuracy(y_pred, y_true):
    # predicted class is the index of highest score in prediction vector (i.e. argmax)
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Define Optimizer Function

In [ ]:
# Stochastic Gradient Descent optimizer
learning_rate = 0.01
optimizer = tf.optimizers.SGD(learning_rate)

# optimization function
def run_optimization(X, y, W, b):
    # wrap computation inside a GradientTape for automatic differentiation
    with tf.GradientTape() as g:
        pred = logistic_regression(X, W, b)
        loss = cross_entropy(pred, y)

    # compute gradients
    gradients = g.gradient(loss, [W, b])

    # update W and b following gradients
    optimizer.apply_gradients(zip(gradients, [W, b]))

## Train Model

In [ ]:
# training parameters
batch_size = 256
training_steps = 1000
display_step = 50

# use tf.data API to shuffle and batch data.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

# weight of shape [784, 10], the 28*28 image features, and total number of classes.
W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")
# bias of shape [10], the total number of classes.
b = tf.Variable(tf.zeros([num_classes]), name="bias")

# run training for the given number of steps
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # run the optimization to update W and b values
    run_optimization(batch_x, batch_y, W, b)

    # display optimization progress every few steps
    if step % display_step == 0:
        pred = logistic_regression(batch_x, W, b)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))
    
    # stop with specific accuracy criteria
    #if acc > CONST:
    #    break        

## Evaluate Model

In [ ]:
# test model on validation set
pred = logistic_regression(x_test, W, b)
print("Test Accuracy: %f" % accuracy(pred, y_test))

## Model Predictions

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# set up matplotlib fig, and size it to fit 3x4 pics
nrows = 1
ncols = 5
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

# predict images from validation set
n_images = 5
test_images = x_test[:n_images]
predictions = logistic_regression(test_images, W, b)

# visualize image and model prediction
for i in range(n_images):
    # set up subplot; subplot indices start at 1
    sp = plt.subplot(nrows, ncols, i + 1, title="pred: %i" % np.argmax(predictions.numpy()[i]))
    sp.axis('Off') # don't show axes (or gridlines)
    plt.imshow(np.reshape(test_images[i], [28, 28]), cmap='gray')

plt.show()